In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import Binarizer
from sklearn.metrics import accuracy_score, precision_score, recall_score,\
confusion_matrix, f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression

import warnings
warnings.filterwarnings('ignore')

Q1. 분류평가: 정확도, 정밀도, 재현율, F1, AUC

In [11]:
titanic_df = pd.read_csv('dataset/tdf.csv')
titanic_df.head()

,Unnamed: 0,survived,pclass,sex,age,sibsp,parch,fare,embarked,age_cat,fare_cat,family
0,0,1,1,0,29.00,0,0,211.3375,0,3,1,0
1,1,1,1,1,0.92,1,2,151.5500,0,1,1,3
2,2,0,1,0,2.00,1,2,151.5500,0,1,1,3
3,3,0,1,1,30.00,1,2,151.5500,0,3,1,3
4,4,0,1,0,25.00,1,2,151.5500,0,3,1,3


In [15]:
y_titanic_df = titanic_df['survived']
X_titanic_df= titanic_df.drop('survived', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, \
                                                    test_size=0.20, random_state=11)

lr_clf = LogisticRegression()

lr_clf.fit(X_train , y_train)
pred = lr_clf.predict(X_test)

In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score,\
confusion_matrix, f1_score

def get_clf_eval(y_test,pred):
    confusion = confusion_matrix(y_test,pred)
    accuracy = accuracy_score(y_test,pred)
    precision = precision_score(y_test,pred)
    recall = recall_score(y_test,pred)
    f1 = f1_score(y_test,pred)
    print('오차행렬')
    print(confusion)
    print('정확도:{0:.4f}, 정밀도:{1:.4f}, 재현율:{2:.4f}, \
    f1 score:{3:.4f}'.format(accuracy,precision,recall,f1))
    
get_clf_eval(y_test,pred)

오차행렬
[[142  14]
 [ 36  70]]
정확도:0.8092, 정밀도:0.8333, 재현율:0.6604,     f1 score:0.7368


In [18]:
pred_proba = lr_clf.predict_proba(X_test)
pred = lr_clf.predict(X_test)
print('pred_proba()결과 shape:{0}'.format(pred_proba.shape))
print('pred_proba array에서 앞 3개만 샘플로 추출 \n:',pred_proba[:3])
print()
print('pred에서 앞 3개만 샘플로 추출 \n:', pred[:3])
print('pred.reshape(-1,1)에서 앞 3개만 샘플로 추출 \n:', pred.reshape(-1,1)[:3])
print()
pred_proba_result = np.concatenate([pred_proba,pred.reshape(-1,1)],axis=1)
pred_proba_result

pred_proba()결과 shape:(262, 2)
pred_proba array에서 앞 3개만 샘플로 추출 
: [[0.88781573 0.11218427]
 [0.74036857 0.25963143]
 [0.39909585 0.60090415]]

pred에서 앞 3개만 샘플로 추출 
: [0 0 1]
pred.reshape(-1,1)에서 앞 3개만 샘플로 추출 
: [[0]
 [0]
 [1]]



array([[0.88781573, 0.11218427, 0.        ],
       [0.74036857, 0.25963143, 0.        ],
       [0.39909585, 0.60090415, 1.        ],
       [0.74525862, 0.25474138, 0.        ],
       [0.88296559, 0.11703441, 0.        ],
       [0.85331352, 0.14668648, 0.        ],
       [0.4984094 , 0.5015906 , 1.        ],
       [0.91657622, 0.08342378, 0.        ],
       [0.80306087, 0.19693913, 0.        ],
       [0.88516027, 0.11483973, 0.        ],
       [0.79303679, 0.20696321, 0.        ],
       [0.88484361, 0.11515639, 0.        ],
       [0.77138275, 0.22861725, 0.        ],
       [0.86884945, 0.13115055, 0.        ],
       [0.56513273, 0.43486727, 0.        ],
       [0.18524202, 0.81475798, 1.        ],
       [0.45405816, 0.54594184, 1.        ],
       [0.81581114, 0.18418886, 0.        ],
       [0.72994976, 0.27005024, 0.        ],
       [0.90267438, 0.09732562, 0.        ],
       [0.8742155 , 0.1257845 , 0.        ],
       [0.85241975, 0.14758025, 0.        ],
       [0.

Q2. 분류평가: 정밀도와 재현율을 인위적으로 조정

In [20]:
X=[[1,-1,2],
  [2,0,0],
  [0,1.1,1.2]]
# threshold(임계값) 기준값 보다 같거나 작으면 0, 크면 1을 반환
binarizer = Binarizer(threshold=1.1)
print(binarizer.fit_transform(X))

[[0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]]


In [21]:
# 생존 확률을 선택해서 임계값 0.5 기준으로 예측
custom_threshold = 0.5
print(pred_proba[0:5])
print()
# 생존 확률 추출 후 2차원 배열로 변환
pred_proba_1 = pred_proba[:,1].reshape(-1,1)
print(pred_proba_1[0:5])
binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_1)
custom_predict = binarizer.transform(pred_proba_1)
custom_predict[0:5]

[[0.88781573 0.11218427]
 [0.74036857 0.25963143]
 [0.39909585 0.60090415]
 [0.74525862 0.25474138]
 [0.88296559 0.11703441]]

[[0.11218427]
 [0.25963143]
 [0.60090415]
 [0.25474138]
 [0.11703441]]


array([[0.],
       [0.],
       [1.],
       [0.],
       [0.]])

In [22]:
def get_clf_eval(y_test,pred):
    confusion = confusion_matrix(y_test,pred)
    accuracy = accuracy_score(y_test,pred)
    precision = precision_score(y_test,pred)
    recall = recall_score(y_test,pred)
    f1 = f1_score(y_test,pred)
    print('오차행렬')
    print(confusion)
    print('정확도:{0:.4f}, 정밀도:{1:.4f}, 재현율:{2:.4f}, \
    f1 score:{3:.4f}'.format(accuracy,precision,recall,f1))
    
get_clf_eval(y_test,custom_predict)

오차행렬
[[142  14]
 [ 36  70]]
정확도:0.8092, 정밀도:0.8333, 재현율:0.6604,     f1 score:0.7368


In [23]:
# 임계값을 내리면 Positive 예측을 많이 할 수 있음
custom_threshold = 0.4
pred_proba_1 = pred_proba[:,1].reshape(-1,1)
binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_1)
custom_predict = binarizer.transform(pred_proba_1)

get_clf_eval(y_test, custom_predict)

오차행렬
[[139  17]
 [ 30  76]]
정확도:0.8206, 정밀도:0.8172, 재현율:0.7170,     f1 score:0.7638


In [24]:
# 테스트를 수행할 모든 임계값을 리스트 객체로 저장
thresholds = [0.4,0.45,0.5,0.55,0.6]

def get_eval_by_threshold(y_test,pred_proba_c1,thresholds):
    # thresholds list 객체내의 값을 차례로 iteration하면서 평가 수행
    for custom_threshold in thresholds:
        binarizer = Binarizer(threshold=custom_threshold).fit(pred_proba_c1)
        custom_predict = binarizer.transform(pred_proba_c1)
        print('임계값:',custom_threshold)
        
        get_clf_eval(y_test,custom_predict)
        print()
get_eval_by_threshold(y_test,pred_proba[:,1].reshape(-1,1),thresholds)


# 레이블 값이 1일때의 예측 확률을 추출
pred_proba_class1 = lr_clf.predict_proba(X_test)[:,1]

# 실제값 데이터셋과 레이블값이 1일때의 예측확률을 precision_recall_curve 인자로 입력
precisions, recalls, thresholds = precision_recall_curve(y_test,pred_proba_class1)
print(thresholds.shape)

thr_index = np.arange(0,thresholds.shape[0],15)
print(thr_index)
print('임계값:',np.round(thresholds[thr_index],2))

# 15 step 단위의 추출된 임계값에 따른 정밀도와 재현율 
print('정밀도:',np.round(precisions[thr_index],3))
print('재현율:',np.round(recalls[thr_index],3))

임계값: 0.4
오차행렬
[[139  17]
 [ 30  76]]
정확도:0.8206, 정밀도:0.8172, 재현율:0.7170,     f1 score:0.7638

임계값: 0.45
오차행렬
[[139  17]
 [ 32  74]]
정확도:0.8130, 정밀도:0.8132, 재현율:0.6981,     f1 score:0.7513

임계값: 0.5
오차행렬
[[142  14]
 [ 36  70]]
정확도:0.8092, 정밀도:0.8333, 재현율:0.6604,     f1 score:0.7368

임계값: 0.55
오차행렬
[[145  11]
 [ 38  68]]
정확도:0.8130, 정밀도:0.8608, 재현율:0.6415,     f1 score:0.7351

임계값: 0.6
오차행렬
[[147   9]
 [ 42  64]]
정확도:0.8053, 정밀도:0.8767, 재현율:0.6038,     f1 score:0.7151

(255,)
[  0  15  30  45  60  75  90 105 120 135 150 165 180 195 210 225 240]
임계값: [0.07 0.1  0.11 0.13 0.15 0.16 0.17 0.2  0.22 0.26 0.31 0.47 0.57 0.67
 0.71 0.75 0.81]
정밀도: [0.416 0.438 0.453 0.476 0.497 0.517 0.552 0.6   0.637 0.683 0.743 0.811
 0.88  0.85  0.889 0.9   0.933]
재현율: [1.    0.991 0.962 0.943 0.915 0.877 0.858 0.849 0.811 0.774 0.736 0.689
 0.623 0.481 0.377 0.255 0.132]


Q3. 회귀평가: MAE, MSE, RMSE

In [27]:
bike_df = pd.read_csv('dataset/bike_train.csv')
bike_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


In [28]:
bike_df['datetime'] = bike_df.datetime.apply(pd.to_datetime)

In [29]:
bike_df['year'] = bike_df.datetime.dt.year
bike_df['month'] = bike_df.datetime.dt.month
bike_df['day'] = bike_df.datetime.dt.day
bike_df['hour'] = bike_df.datetime.dt.hour
bike_df.drop('datetime', axis=1, inplace=True)
bike_df.head()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,year,month,day,hour
0,1,0,0,1,9.84,14.395,81,0.0,3,13,16,2011,1,1,0
1,1,0,0,1,9.02,13.635,80,0.0,8,32,40,2011,1,1,1
2,1,0,0,1,9.02,13.635,80,0.0,5,27,32,2011,1,1,2
3,1,0,0,1,9.84,14.395,75,0.0,3,10,13,2011,1,1,3
4,1,0,0,1,9.84,14.395,75,0.0,0,1,1,2011,1,1,4


In [30]:
bike_df.drop(['casual', 'registered'], axis=1, inplace=True)

In [34]:
def rmsle(y,pred):
    log_y = np.log1p(y)
    log_pred = np.log1p(pred)
    squared_error = (log_y - log_pred)**2
    rmsle = np.sqrt(np.mean(squared_error))
    return rmsle

def rmse(y,pred):
    return np.sqrt(mean_squared_error(y,pred))

def evaluate_regr(y,pred):
    rmsle_val = rmsle(y,pred)
    rmse_val = rmse(y,pred)
    mae_val = mean_absolute_error(y,pred)
    print('MAE:{0:.4f}, MSE:{1:.4f}, RMSE:{2:.4f}'.format(mae_val,(rmse_val)**2, rmse_val))

In [35]:
y_target = bike_df['count']
X_features = bike_df.drop(['count'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(X_features,y_target,\
                                                    test_size=0.3,random_state=0)
lr_reg = LinearRegression()
lr_reg.fit(X_train,y_train)
pred = lr_reg.predict(X_test)
evaluate_regr(y_test,pred)

MAE:105.9244, MSE:19852.6941, RMSE:140.8996


Q4. 회귀평가: R2_score, R_square

Q5. 교차검증: cross_val_score

Q6. 교차검증, 파라미터튜닝: GridSearchCV

Q7. RMSLE 평가

Q8. 예측값 혼합으로 평가